In [18]:
import pandas as pd
import json
import os
import re

def safe_get(data_list, key='', default=None):
    if not data_list:
        return default
    data = data_list[0]
    return data.get(key, default) if isinstance(data, dict) else default

def get_varietal_label(data_list, default=''):
    if data_list and 'varietalDesignationCode' in data_list[0]:
        return data_list[0]['varietalDesignationCode'].get('label', default)
    return default

def get_label(data, default=''):
    return data.get('label', default) if isinstance(data, dict) else default

def extract_appellation(product_benefits):
    for benefit in product_benefits:
        for text in benefit.get('text', []):
            data = text.get('data', '')
            match = re.search(r'Appellation:\s*(.*)', data)
            if match:
                return match.group(1).strip()
    return ''

# Set the directory containing the JSON files
directory_path = os.getenv('JSON_DIRECTORY_PATH', r'D:\1800spirits-salsify-data-load\input_folders\winefiles')

# Set the output directory
output_directory = r'D:\1800spirits-salsify-data-load\Output_files'
os.makedirs(output_directory, exist_ok=True)

# Initialize an empty list to collect rows
rows = []

# Iterate over all JSON files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        
        # Load each JSON file with utf-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)['data']
        
        # Extract data for each item in the JSON file
        for item in data:
            vineyard_region = item.get('vineyardRegion', {})
            product_benefits = item.get('productBenefits', [])
            net_content = item.get('netContent', [{}])[0] if item.get('netContent') else {}
            row = {
                'GTIN': item.get('gtin', ''),
                'Manufacturer Name': item.get('manufacturerNameText', ''),
                'Brand': item.get('brandText', ''),
                'Volume': net_content.get('data', ''),
                'Volume Code': net_content.get('expressedIn', {}).get('code', ''),
                'namePublicLong': safe_get(item.get('namePublicLong', [{}]), 'data', ''),
                'namePublicShort': safe_get(item.get('namePublicShort', [{}]), 'data', ''),
                'functionalName': safe_get(item.get('functionalName', [{}]), 'data', ''),
                'typePackaging': item.get('typePackaging', {}).get('code', ''),
                'useableUnits': item.get('useableUnits', ''),
                'ratioAlcohol': item.get('ratioAlcohol', ''),
                'packagingInformationList': get_label(item.get('packagingInformationList', [{}]), ''),
                'countryOfOriginList': get_label(item.get('countryOfOriginList', [{}]), 'USA'),
                'kind': item.get('kind', {}).get('label', ''),
                'varietalDesignationList': get_varietal_label(item.get('varietalDesignationList', [])),
                'productbenefits': extract_appellation(product_benefits),
                'vineyardRegion': get_label(vineyard_region, ''),
                'vintage': item.get('vintage', '')
            }
            rows.append(row)

# Create a DataFrame from the list of rows
df = pd.DataFrame(rows, columns=[
    'GTIN',
    'Manufacturer Name',
    'Brand',
    'Volume',
    'Volume Code',
    'namePublicLong',
    'namePublicShort',
    'functionalName',
    'typePackaging',
    'useableUnits',
    'ratioAlcohol',
    'packagingInformationList',
    'countryOfOriginList',
    'kind',
    'varietalDesignationList',
    'productbenefits',
    'vineyardRegion',
    'vintage'
])

# Save the DataFrame to a CSV file
output_csv_path = os.path.join(output_directory, 'winefiles-8-July-27files.csv')
df.to_csv(output_csv_path, index=False)

print(f"Data extraction complete. CSV file created at '{output_csv_path}'.")


Data extraction complete. CSV file created at 'D:\1800spirits-salsify-data-load\Output_files\winefiles-8-July-27files.csv'.
